In [1]:
import os

import numpy as np
import pandas as pd

In [2]:
os.getcwd()

'/home/anthony/summer-bursary-2018/bacteria'

In [3]:
os.listdir('data')

['Classific.py',
 'graphs.pdf',
 'growth_stages.csv',
 '16ms_32ms_growth_phase_spectra.csv',
 'PC6allwavelengths.png',
 'wavelengths.csv',
 '16_ms_lag_codes.csv',
 'bacteria.csv']

In [4]:
df = pd.read_csv('data/bacteria.csv', header=[0, 1, 2], index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1043 entries, 0 to 1042
Columns: 135 entries, (lag, bc, 01) to (stat, se, 07)
dtypes: float64(135)
memory usage: 1.1 MB


# Preprocessing

In [5]:
X = df.reorder_levels([1, 0, 2], axis=1).sort_index(axis=1).T
X.head()

0         1         2         3     \
species growth_phase replicate                                           
bc      lag          01         0.000000  1.560060  0.715025 -0.520060   
                     02         0.622353 -2.115990  0.062234  2.178335   
                     03         0.064528  0.645279 -1.032450 -2.064991   
                     04        -1.094650 -2.554170 -0.912210  0.851433   
                     05        -1.492810 -0.559800  2.923387  3.047967   

                                    4         5             6         7     \
species growth_phase replicate                                               
bc      lag          01         0.130014 -2.080180  0.000000e+00  1.040112   
                     02         1.867163  1.120305  8.713430e-01 -1.369260   
                     03        -1.419711  1.032515 -2.000491e+00 -0.903453   
                     04         3.709871  1.520432  8.514440e-01 -2.067810   
                     05         2.985793  2.985763 -3.590000e-07 -0.995270   

                                    8         9       ...         1033  \
species growth_phase replicate                        ...                
bc      lag          01         1.235136 -0.195020    ...     1.170374   
                     02        -1.929420  0.684622    ...    -0.373450   
                     03         2.968462  1.032499    ...     3.097847   
                     04        -1.581270  0.304085    ...     4.257782   
                     05        -0.933060 -1.990480    ...     4.479034   

                                    1034      1035      1036      1037  \
species growth_phase replicate                                           
bc      lag          01         2.730861  3.835922  1.755222  1.235332   
                     02        -0.187040 -2.302970  0.124451  0.809098   
                     03         4.775991  4.775919  4.259427  2.129823   
                     04         3.284733  0.851676  2.493739  3.467134   
                     05         2.488391  3.546049  1.057399  3.981489   

                                    1038      1039      1040      1041  \
species growth_phase replicate                                           
bc      lag          01         4.355984  0.520024  0.715032 -0.325010   
                     02         1.058059  2.676111  0.622357  0.560118   
                     03         0.064477 -2.516599  0.516229 -0.903391   
                     04         5.900169  1.520354  2.067671  0.121629   
                     05         6.469838 -0.933000  0.062199 -0.435400   

                                    1042  
species growth_phase replicate            
bc      lag          01        -0.520030  
                     02        -0.435650  
                     03         1.096991  
                     04         1.094670  
                     05         1.430629  

[5 rows x 1043 columns]

In [6]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False)
growth_phases = X.reset_index()['growth_phase']
growth_phases = growth_phases.values  # get the numpy array
growth_phases = growth_phases.reshape(-1, 1)

print(growth_phases[:5])

ohe.fit(growth_phases)
growth_phases = ohe.transform(growth_phases)

print(growth_phases[:5])

[['lag']
 ['lag']
 ['lag']
 ['lag']
 ['lag']]
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [7]:
X['lag'] = growth_phases[:, 0]
X['log'] = growth_phases[:, 1]
X['stat'] = growth_phases[:, 2]

X.head()

0         1         2         3  \
species growth_phase replicate                                           
bc      lag          01         0.000000  1.560060  0.715025 -0.520060   
                     02         0.622353 -2.115990  0.062234  2.178335   
                     03         0.064528  0.645279 -1.032450 -2.064991   
                     04        -1.094650 -2.554170 -0.912210  0.851433   
                     05        -1.492810 -0.559800  2.923387  3.047967   

                                       4         5             6         7  \
species growth_phase replicate                                               
bc      lag          01         0.130014 -2.080180  0.000000e+00  1.040112   
                     02         1.867163  1.120305  8.713430e-01 -1.369260   
                     03        -1.419711  1.032515 -2.000491e+00 -0.903453   
                     04         3.709871  1.520432  8.514440e-01 -2.067810   
                     05         2.985793  2.985763 -3.590000e-07 -0.995270   

                                       8         9  ...       1036      1037  \
species growth_phase replicate                      ...                        
bc      lag          01         1.235136 -0.195020  ...   1.755222  1.235332   
                     02        -1.929420  0.684622  ...   0.124451  0.809098   
                     03         2.968462  1.032499  ...   4.259427  2.129823   
                     04        -1.581270  0.304085  ...   2.493739  3.467134   
                     05        -0.933060 -1.990480  ...   1.057399  3.981489   

                                    1038      1039      1040      1041  \
species growth_phase replicate                                           
bc      lag          01         4.355984  0.520024  0.715032 -0.325010   
                     02         1.058059  2.676111  0.622357  0.560118   
                     03         0.064477 -2.516599  0.516229 -0.903391   
                     04         5.900169  1.520354  2.067671  0.121629   
                     05         6.469838 -0.933000  0.062199 -0.435400   

                                    1042  lag  log  stat  
species growth_phase replicate                            
bc      lag          01        -0.520030  1.0  0.0   0.0  
                     02        -0.435650  1.0  0.0   0.0  
                     03         1.096991  1.0  0.0   0.0  
                     04         1.094670  1.0  0.0   0.0  
                     05         1.430629  1.0  0.0   0.0  

[5 rows x 1046 columns]

In [8]:
y = X.reset_index()['species']
y.head()

0    bc
1    bc
2    bc
3    bc
4    bc
Name: species, dtype: object

In [9]:
y.describe()

count     135
unique      6
top        bc
freq       36
Name: species, dtype: object

In [10]:
print(y)

0      bc
1      bc
2      bc
3      bc
4      bc
5      bc
6      bc
7      bc
8      bc
9      bc
10     bc
11     bc
12     bc
13     bc
14     bc
15     bc
16     bc
17     bc
18     bc
19     bc
20     bc
21     bc
22     bc
23     bc
24     bc
25     bc
26     bc
27     bc
28     bc
29     bc
       ..
105    sa
106    sa
107    sa
108    sa
109    sa
110    sa
111    sa
112    sa
113    sa
114    sa
115    sa
116    sa
117    sa
118    se
119    se
120    se
121    se
122    se
123    se
124    se
125    se
126    se
127    se
128    se
129    se
130    se
131    se
132    se
133    se
134    se
Name: species, Length: 135, dtype: object


In [11]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(X_scaled.mean(), X_scaled.std())
print(X_scaled.shape)
print(X[:5])

2.6417033940517855e-19 1.0
(135, 1046)
                                       0         1         2         3  \
species growth_phase replicate                                           
bc      lag          01         0.000000  1.560060  0.715025 -0.520060   
                     02         0.622353 -2.115990  0.062234  2.178335   
                     03         0.064528  0.645279 -1.032450 -2.064991   
                     04        -1.094650 -2.554170 -0.912210  0.851433   
                     05        -1.492810 -0.559800  2.923387  3.047967   

                                       4         5             6         7  \
species growth_phase replicate                                               
bc      lag          01         0.130014 -2.080180  0.000000e+00  1.040112   
                     02         1.867163  1.120305  8.713430e-01 -1.369260   
                     03        -1.419711  1.032515 -2.000491e+00 -0.903453   
                     04         3.709871  1.520432  

In [12]:
from sklearn.utils import shuffle

X, y = shuffle(X, y)

print(X[:5])
print(y[:5])

                                       0         1         2         3  \
species growth_phase replicate                                           
bc      stat         03        -0.279776  0.419667 -0.419668  0.279793   
                     06         0.361383 -0.542070 -0.782990  0.361400   
ec      lag          02         0.786579 -1.114310  0.721023  1.638780   
se      stat         02         1.283523  0.192528  1.219347 -3.337340   
        log          05         0.787276  0.242244  0.242238  1.756314   

                                       4         5         6         7  \
species growth_phase replicate                                           
bc      stat         03         2.378291  1.049258  0.489651  6.155592   
                     06         1.325136  1.264897 -0.180700  1.867256   
ec      lag          02         1.507702  0.262210  0.524415 -0.983280   
se      stat         02         1.604498 -1.732870  0.898522  0.834347   
        log          05        -0.181

# PCA + SVM

In [21]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.99, svd_solver='full')
pca.fit(X_scaled)

X_pca = pca.transform(X_scaled)
print(X_pca.shape)

(135, 75)


In [14]:
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score, GridSearchCV

param_grid = {
    'kernel': ['linear', 'rbf'],
    'gamma': [10 ** -n for n in range(10)],
    'C': [10 ** n for n in range(-9, 2)]
}

cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=20)
clf = SVC()

grid_search = GridSearchCV(clf, param_grid, cv=cv, verbose=1, n_jobs=-1)
grid_search.fit(X, y)

print(grid_search.best_score_)
print(grid_search.best_params_)

Fitting 60 folds for each of 264 candidates, totalling 15840 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 990 tasks      | elapsed:   10.6s
[Parallel(n_jobs=-1)]: Done 2490 tasks      | elapsed:   25.2s
[Parallel(n_jobs=-1)]: Done 4590 tasks      | elapsed:   45.6s
[Parallel(n_jobs=-1)]: Done 6917 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 8083 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 8845 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-1)]: Done 9689 tasks      | elapsed: 10.8min
[Parallel(n_jobs=-1)]: Done 10635 tasks      | elapsed: 15.0min
[Parallel(n_jobs=-1)]: Done 11712 tasks      | elapsed: 19.6min
[Parallel(n_jobs=-1)]: Done 12844 tasks      | elapsed: 24.2min
[Parallel(n_jobs=-1)]: Done 14087 tasks      | elapsed: 29.6min
[Parallel(n_jobs=-1)]: Done 15594 tasks      | elapsed: 35.7min
[Parallel(n_jobs=-1)]: Done 15833 out of 15840 | elapsed: 36.6min remaining:    1.0s
[Parallel(n_jobs=

0.5603703703703704
{'C': 0.0001, 'gamma': 1, 'kernel': 'linear'}


In [15]:
%%time

cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=20)
scores = cross_val_score(grid_search.best_estimator_, X_pca, y, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.27 (+/- 0.02)
CPU times: user 1.05 s, sys: 0 ns, total: 1.05 s
Wall time: 1.05 s


In [16]:
%%time

cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=20)
scores = cross_val_score(grid_search.best_estimator_, X, y, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.57 (+/- 0.13)
CPU times: user 8min 31s, sys: 168 ms, total: 8min 31s
Wall time: 8min 31s


# Random Forest with Decision Stumps

In [18]:
%%time

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=1024, max_depth=1)
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=20)
scores = cross_val_score(clf, X_pca, y, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.27 (+/- 0.02)
CPU times: user 47.6 s, sys: 52 ms, total: 47.7 s
Wall time: 47.6 s


In [17]:
%%time

clf = RandomForestClassifier(n_estimators=1024, max_depth=1)
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=20)
scores = cross_val_score(clf, X, y, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.39 (+/- 0.10)
CPU times: user 55.7 s, sys: 16 ms, total: 55.8 s
Wall time: 55.8 s


# AdaBoost

In [22]:
%%time

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),
                        n_estimators=1024)
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=20)
scores = cross_val_score(clf, X_pca, y, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.21 (+/- 0.09)
CPU times: user 1min 47s, sys: 70.6 ms, total: 1min 47s
Wall time: 1min 47s


In [23]:
%%time

clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),
                        n_estimators=1024)
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=20)
scores = cross_val_score(clf, X, y, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.28 (+/- 0.12)
CPU times: user 9min 12s, sys: 568 ms, total: 9min 12s
Wall time: 9min 12s
